In [1]:
import xarray as xr
from datetime import date
import dl_tools

# Kringla

Load and prep the data.txt from the datalogger.

In [2]:
path = '/Users/xedhjo/Downloads/drive-download-20230511T124300Z-001/DATA.TXT'

In [3]:
ds = dl_tools.load_sb(path)                                                      # Loading the data
ds = dl_tools.fix_standard_attrs(ds)                                             # Setting attributes and names for the standard variables
ds = dl_tools.fix_airmar(ds)                                                     # Setting attributes and names for the Airmar variables
ds = dl_tools.fix_dcps(ds)                                                       # Setting attributes and names for the DCPS variables
ds = dl_tools.fix_aadi(ds)                                                       # Setting attributes and names for the AADI Conductivity variables
ds = ds.sortby('time').sel(time=slice('2023-03-06 11:20','2023-05-04 04:40'))    # Sorting in time, and selecting the time for the mission

Checking lines in input file:   0%|          | 0/8533 [00:00<?, ?it/s]

8531 good measurements found in input file
2 bad lines found in input file



#### Assigning the metadata

In [4]:
ds.attrs['title']                   = 'SB Kringla Datalogger'
ds.attrs['summary']                 = 'Datalogger output from Sailbuoy Kringla'
ds.attrs['conventions']             = 'CF-1.6, ACDD-1.3'
ds.attrs['creator_name']            = 'Johan Edholm'
ds.attrs['platform']                = 'Uncrewed surface vehicle'
ds.attrs['instrument_name']         = 'Kringla'
ds.attrs['instrument_id']           = '1812'
ds.attrs['institution']             = 'University of Gothenburg'
ds.attrs['creator_email']           = 'johan.edholm@gu.se'
ds.attrs['area']                    = 'Southern Ocean'
ds.attrs['project']                 = 'QUICCHE'
ds.attrs['id']                      = 'SB Kringla Datalogger QUICCHE'
ds.attrs['instrument']              = 'Offshore Sensing AS Sailbuoy'
ds.attrs['owner']                   = 'University of Gothenburg'
ds.attrs['contact']                 = 'sebastiaan.swart@marine.gu.se'
ds.attrs['note']                    = 'Apologies to anyone that is forced to work with this data...'
ds.attrs['processing_date']         = str(date.today())
ds.attrs['processing_level']        = '2'
ds.attrs['time_coverate_start']     = str(ds.time[0].values)
ds.attrs['time_coverate_end']       = str(ds.time[-1].values)
ds.attrs['geospatial_lat_min']      = str(ds.latitude.min().values)
ds.attrs['geospatial_lat_max']      = str(ds.latitude.max().values)
ds.attrs['geospatial_lon_min']      = str(ds.longitude.min().values)
ds.attrs['geospatial_lon_max']      = str(ds.longitude.max().values)

In [7]:
ds.to_netcdf('/Users/xedhjo/Documents/Projects/QUICCHE/data/Kringla_QUICCHE_data.nc')

# PimPim

In [5]:
path = '/Users/xedhjo/Documents/Projects/QUICCHE/data/sailbuoy/PimPim/full/DATA.TXT'

In [6]:
#path = '/Users/xedhjo/Documents/Projects/SO-CHIC/data/DATA.TXT'

ds = dl_tools.load_sb(path)                                          # Loading the data
ds = dl_tools.fix_standard_attrs(ds)                                 # Setting attributes and names for the standard variables
ds = dl_tools.fix_FT(ds)                                             # Setting attributes and names for the Airmar variables
ds = dl_tools.fix_dcps(ds)                                           # Setting attributes and names for the DCPS variables
ds = dl_tools.fix_legato(ds)                                         # Setting attributes and names for the AADI Conductivity variables
ds = dl_tools.fix_T9602(ds)                                          # Setting attributes and names for the AADI Conductivity variables
ds = ds.sortby('time').where(ds.ssc > 0,drop=True)

Checking lines in input file:   0%|          | 0/8995 [00:00<?, ?it/s]

8995 good measurements found in input file
0 bad lines found in input file



In [12]:
ds.attrs['title']                   = 'SB PimPim Datalogger'
ds.attrs['summary']                 = 'Datalogger output from Sailbuoy PimPim'
ds.attrs['conventions']             = 'CF-1.6, ACDD-1.3'
ds.attrs['creator_name']            = 'Johan Edholm'
ds.attrs['platform']                = 'Uncrewed surface vehicle'
ds.attrs['instrument_name']         = 'PimPim'
ds.attrs['instrument_id']           = '2326'
ds.attrs['institution']             = 'University of Gothenburg'
ds.attrs['creator_email']           = 'johan.edholm@gu.se'
ds.attrs['area']                    = 'Cape Cauldron'
ds.attrs['project']                 = 'QUICCHE'
ds.attrs['id']                      = 'SB PimPim Datalogger QUICCHE'
ds.attrs['instrument']              = 'Offshore Sensing AS Sailbuoy'
ds.attrs['owner']                   = 'University of Gothenburg'
ds.attrs['contact']                 = 'sebastiaan.swart@marine.gu.se'
ds.attrs['note']                    = 'Apologies to anyone that is forced to work with this data...'
ds.attrs['processing_date']         = str(date.today())
ds.attrs['processing_level']        = '2'
ds.attrs['time_coverate_start']     = str(ds.time[0].values)
ds.attrs['time_coverate_end']       = str(ds.time[-1].values)
ds.attrs['geospatial_lat_min']      = str(ds.latitude.min().values)
ds.attrs['geospatial_lat_max']      = str(ds.latitude.max().values)
ds.attrs['geospatial_lon_min']      = str(ds.longitude.min().values)
ds.attrs['geospatial_lon_max']      = str(ds.longitude.max().values)

In [15]:
ds.to_netcdf(path[:-3] + 'nc')